<a href="https://colab.research.google.com/github/Hero0963/data_science_camp_level2/blob/main/task03/dscamp_lv2_task03_hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#練習：在前述的練習中，你已經利用爬蟲成功的收集到了來自 Yahoo 奇摩電影中「本週新片」、「上映中」與「即將上映」前三個分頁中的電影，包含「中、英文名稱」、「上映日期」和「期待度」資訊。接下來請你將這些資料整理成一個 DataFrame 並且將資料存到到 SQLite 資料庫（名稱欄位自訂）中。

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

conn = sqlite3.connect("movies.db")
 
urls = ["https://movies.yahoo.com.tw/movie_thisweek.html", "https://movies.yahoo.com.tw/movie_intheaters.html", "https://movies.yahoo.com.tw/movie_comingsoon.html"]
table_name_list = ["thisweek", "intheaters", "comingsoon"]

search_pages = 3



for idx, url in enumerate(urls):
    # print(url)
    # print()
    pd_movies_info_merge = pd.DataFrame([], columns=['中文名稱','英文名稱','上映日期','期待度'])
    for i in range(1, search_pages + 1):
        # print("page = ", i)
        new_tab_url = url + "?page=" + str(i)
        response = requests.get(url=new_tab_url)
        soup = BeautifulSoup(response.text, 'lxml')
        info_items = soup.find_all('div', 'release_info')
        movies_info = []
        for item in info_items:

            name= item.find('div', 'release_movie_name').a.text.strip()
            english_name = item.find('div', 'en').a.text.strip()
            release_time = item.find('div', 'release_movie_time').text.split('：')[-1].strip()
            
            expectation = ""
            expectation_info = item.find('div', 'leveltext')
            if expectation_info:
                expectation = item.find('div', 'leveltext').span.text.strip()

            movies_info.append([name, english_name, release_time, expectation])


        pd_movies_info = pd.DataFrame(movies_info, columns=['中文名稱','英文名稱','上映日期','期待度'])
        # display(pd_movies_info)

        pd_movies_info_merge = pd.concat([pd_movies_info_merge, pd_movies_info], axis=0)

        if pd_movies_info.empty:
            break

    # pd_movies_info_merge.set_index('中文名稱', inplace=True)
    # print("pd_movies_info_merge = ", pd_movies_info_merge)
    table_name = table_name_list[idx]
    pd_movies_info_merge.to_sql(table_name, conn, if_exists='replace', index=False)

print("finished")

finished


In [2]:
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('thisweek',), ('intheaters',), ('comingsoon',)]


In [3]:
for table_name in table_name_list:
    df = pd.read_sql_query("SELECT * from " + table_name, conn)
    print(table_name)
    print()
    display(df)

thisweek



,中文名稱,英文名稱,上映日期,期待度
0,玩命關頭X,Fast X,2023-05-17,91%
1,我的2年妓女生涯,La Maison,2023-05-19,88%
2,消失的紫斑蝶,,2023-05-19,86%
3,收陰機,Sound of Silence,2023-05-19,100%
4,烈火焚身,Incendies,2023-05-19,100%
5,摘星甜點王,Sugar and Stars,2023-05-19,100%
6,非關男孩,About a Boy,2023-05-19,100%
7,擒慾女王,Queenie Classy Lusty Baby,2023-05-19,100%


intheaters



,中文名稱,英文名稱,上映日期,期待度
0,我的幸福婚約,As Long As We Both Shall Live,2023-05-12,100%
1,神鬼聖戰,The Devil Conspiracy,2023-05-12,100%
2,湯道,Yudo,2023-05-12,100%
3,油麻菜籽,Ah Fei,2023-05-12,89%
4,龍馬精神,Ride On,2023-05-12,61%
5,正宗巨齒鯊,The Black Demon,2023-05-12,98%
6,金之國水之國,Gold Kingdom and Water Kingdom,2023-05-12,100%
7,陰影下的她,Next Sohee,2023-05-12,100%
8,Anne的多重恐懼,Faces of Anne,2023-05-12,100%
9,玫瑰母親,Mother and Son,2023-05-12,100%


comingsoon



,中文名稱,英文名稱,上映日期,期待度
0,小美人魚,Little Mermaid,2023-05-25,24%
1,色局追兇,360,2023-05-26,67%
2,厲嬰魂,The Unborn Soul,2023-05-26,80%
3,駭人骨娃娃,The Communion Girl,2023-05-26,75%
4,永遠的好麻吉,Dostojee,2023-05-26,100%
5,坎達哈行動,Kandahar,2023-05-26,100%
6,愛的密語,Safe Word,2023-05-26,100%
7,當男人戀愛時,Man in Love,2023-05-26,97%
8,燃起希望之火,Harka,2023-05-26,100%
9,禁忌的畫像：卡拉瓦喬之影,Caravaggio’S Shadow,2023-05-26,100%


In [4]:
conn.close()